7.22 Dota version

In [ ]:
import pandas as pd
import numpy as np

import itertools

import urllib.request
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Counter Matchup Data

##### Web scrap from Dotapedia

###### Function to remove duplicates and keep the list order 
http://www.martinbroadhurst.com/removing-duplicates-from-a-list-while-preserving-order-in-python.html


In [ ]:
def unique(sequence):
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

###### Get all heroes names

##### Function to process data from links on https://dota2.gamepedia.com/Matchups

In [ ]:
def get_heroes_matchup_info(hero_name, heroes_mentioned):
    
    # Part 1: clean up the information collected from the webite  
    # filter non type element in the list
    heroes_mentioned = list(filter(None, heroes_mentioned)) 

    # cut short the list to only relevant heroes
    heroes_mentioned_clean = heroes_mentioned[heroes_mentioned.index('/index.php?title=' + hero_name + \
                                                                     '/Counters&action=edit&section=1'): 
                                              heroes_mentioned.index('/Dota_2_Wiki:About') -1]

    # remove '/' i the list
    heroes_mentioned_clean = [heroes_mentioned[i].strip('/') for i in 
                              range(len(heroes_mentioned))]    

    # remove duplicates in the list    
    heroes_mentioned_clean = unique(heroes_mentioned_clean)

    # remove skills and items (can think more about this later)
    heroes_mentioned_clean1 = []

    for i in range(len(heroes_mentioned_clean)):
        if '#' not in heroes_mentioned_clean[i]:
            heroes_mentioned_clean1.append(heroes_mentioned_clean[i])

    # filter only heroes        
    heroes_mentioned_clean2 = [heroes_mentioned_clean1[i] for i in 
                               range(len(heroes_mentioned_clean1)) if 
                               heroes_mentioned_clean1[i] in heroes_name_list]

    # Part 2: populate the hero's matchup info   
    # In the page, heroes that counters the hero well is in red  
    heroes_countered1 = []

    productDivs = soup.findAll('div', attrs={'style': 'margin-bottom:5px; box-shadow:0px 0px 2px 4px red;'})
    for div in productDivs:
        heroes_countered1.append(div.find('a')['href'].strip('/'))

    # In the page, heroes that the hero counters well is in green       
    heroes_counter1 = []

    productDivs_chartreuse = soup.findAll('div', attrs={'style': 'margin-bottom:5px; box-shadow:0px 0px 2px 4px chartreuse;'})
    productDivs_green = soup.findAll('div', attrs={'style': 'margin-bottom:5px; box-shadow:0px 0px 2px 4px green;'})
    productDivs = productDivs_chartreuse + productDivs_green
    for div in productDivs:
        heroes_counter1.append(div.find('a')['href'].strip('/'))

    # In the page, heroes good with are list in blue     
    heroes_good_with1 = []

    productDivs = soup.findAll('div', attrs={'style': 'margin-bottom:5px; box-shadow:0px 0px 2px 4px skyblue;'})
    for div in productDivs:
        heroes_good_with1.append(div.find('a')['href'].strip('/'))

    # In the page, great countered heroes are mentioned in red, then there are some other heroes mentioned before the page moves on to the next section
    # last hero mentioned in red 
    last_red = heroes_mentioned_clean2.index(heroes_countered1[-1])
    # first hero mentioned in green
    first_green = heroes_mentioned_clean2.index(heroes_counter1[0])
    # last hero mentioned in green
    last_green = heroes_mentioned_clean2.index(heroes_counter1[-1])

    # first hero mentioned in blue
    first_blue = heroes_mentioned_clean2.index(heroes_good_with1[0])
    # last hero mentioned in blue
    last_blue = heroes_mentioned_clean2.index(heroes_good_with1[-1])

    # Create list of heroes matchups    
    # Create countered level 2
    if last_red+1 == first_green:
        heroes_countered2 = []
    else:     
        heroes_countered2 = heroes_mentioned_clean2[last_red+1 : first_green]

    # Create counter level 2
    if last_green+1 == first_blue: 
        heroes_counter2 = []
    else:
        heroes_counter2 = heroes_mentioned_clean2[last_green+1 : first_blue]

    # Create good with level 2
    heroes_good_with2 = heroes_mentioned_clean2[last_blue+1 :]

    # Part 3: export the info 
    # to an object
    heroes_matchup_info = {'heroes_countered1': heroes_countered1, 
                           'heroes_countered2': heroes_countered2,
                           'heroes_counter1': heroes_counter1, 
                           'heroes_counter2': heroes_counter2,
                           'heroes_good_with1': heroes_good_with1,
                           'heroes_good_with2': heroes_good_with2}
    
    return heroes_matchup_info

In [ ]:
with urlopen('https://raw.githubusercontent.com/dqa611/Dota2_analysis/master/dota2_heroes.csv') as csv_file:
    heroes_name_fixed = pd.read_csv(csv_file)

In [ ]:
heroes_name_list = heroes_name_fixed['Heroes'].tolist()

##### Get all the link the page

In [ ]:
r = requests.get('https://dota2.gamepedia.com/Matchups')
soup = BeautifulSoup(r.text)

base_url = 'https://dota2.gamepedia.com/Matchups'
heroes_counters_link = []

for link in soup.find_all('a'):
    url = link.get('href')
    if url and '/Counters' in url:
        heroes_counters_link.append(url)
        
heroes_counters_link = list(sorted(set(heroes_counters_link)))

In [ ]:
heroes_matchup_dict = {}

for i in range(len(heroes_counters_link)):
    r = requests.get('https://dota2.gamepedia.com' + heroes_counters_link[i])
    soup = BeautifulSoup(r.text)

    hero_name = heroes_counters_link[i].strip('/')[:-9]

    heroes_mentioned = []
    for link in soup.findAll('a'):
        heroes_mentioned.append(link.get('href')) 
    
    heroes_matchup_dict[hero_name] = get_heroes_matchup_info(hero_name, heroes_mentioned)

###### Save the Data to Json file

In [ ]:
import json
json = json.dumps(heroes_matchup_dict)

f = open("C:/Users/anhqu/OneDrive/Desktop/QAs/QA github folder/dotapedia_matchup_dict.json","w")
f.write(json)
f.close()

In [ ]:
import json
with open("C:/Users/anhqu/OneDrive/Desktop/QAs/QA github folder/dotapedia_matchup_dict.json") as json_file:
    heroes_matchup_dict = json.load(json_file)

### Create Matchup Pair Data

In [ ]:
heroes_matchup_pair = list(itertools.permutations(heroes_name_list,2))

## Matchup Score

In [ ]:
matchup_score = []
for i in range(len(heroes_matchup_pair)):    
    matchup_score.append(list([heroes_matchup_pair[i],0]))

In [ ]:
for i in range(len(heroes_matchup_pair)):

    hero1_in_pair = heroes_matchup_pair[i][0]
    hero2_in_pair = heroes_matchup_pair[i][1]
    
    list_counter_countered = list(itertools.chain(*[heroes_matchup_dict[hero1_in_pair]['heroes_countered2'],
                                                    heroes_matchup_dict[hero1_in_pair]['heroes_counter2'],
                                                    heroes_matchup_dict[hero1_in_pair]['heroes_countered1'],
                                                    heroes_matchup_dict[hero1_in_pair]['heroes_counter1']]
                                                 ))

# 0 means the 2 heores have ways to counter each other
# negative means hero 1 in the pair is countered by hero 2
# positive means hero 1 in the pair counters hero 2

    if hero2_in_pair not in list_counter_countered:
        matchup_score[i][1] = 0
    if hero2_in_pair in heroes_matchup_dict[hero1_in_pair]['heroes_countered2']:
        matchup_score[i][1] = -.5
    if hero2_in_pair in heroes_matchup_dict[hero1_in_pair]['heroes_counter2']:
        matchup_score[i][1] = .5
    if hero2_in_pair in heroes_matchup_dict[hero1_in_pair]['heroes_countered1']:
        matchup_score[i][1] = -1
    if hero2_in_pair in heroes_matchup_dict[hero1_in_pair]['heroes_counter1']:
        matchup_score[i][1] = 1

In [ ]:
matchup_score1 = matchup_score[:]

for i in range(len(matchup_score1)):

    hero1_in_pair = heroes_matchup_pair[i][0]
    hero2_in_pair = heroes_matchup_pair[i][1]

    similar_pairs = [y for y in matchup_score if hero1_in_pair in y[0] and hero2_in_pair in y[0]]    

    heroes_pair_1 = similar_pairs[0]
    heroes_pair_2 = similar_pairs[1]

    if abs(heroes_pair_1[1]) > abs(heroes_pair_2[1]):
        matchup_score1[matchup_score1.index(heroes_pair_2)][1] = -heroes_pair_1[1]
    elif abs(heroes_pair_1[1]) < abs(heroes_pair_2[1]):
        matchup_score1[matchup_score1.index(heroes_pair_1)][1] = -heroes_pair_2[1]

In [ ]:
import json
json = json.dumps(matchup_score1)

f = open("C:/Users/anhqu/OneDrive/Desktop/QAs/QA github folder/heroes_matchup_score.json","w")
f.write(json)
f.close()

## Good With Score

In [ ]:
good_with_score = []
for i in range(len(heroes_matchup_pair)):    
    good_with_score.append(list([heroes_matchup_pair[i],0]))

In [ ]:
import json
json = json.dumps(good_with_score1)

f = open("C:/Users/anhqu/OneDrive/Desktop/QAs/QA github folder/heroes_good_with_score.json","w")
f.write(json)
f.close()

In [ ]:
for i in range(len(heroes_matchup_pair)):

    hero1_in_pair = heroes_matchup_pair[i][0]
    hero2_in_pair = heroes_matchup_pair[i][1]
    
    list_good_with = list(itertools.chain(*[heroes_matchup_dict[hero1_in_pair]['heroes_good_with1'],
                                                    heroes_matchup_dict[hero1_in_pair]['heroes_good_with2']
                                           ]))

# 0 means the 2 heroes don't significantly go well together
# 0.5 means OK
# 1 means combo buddy                                        

    if hero2_in_pair not in list_good_with:
        good_with_score[i][1] = 0
    if hero2_in_pair in heroes_matchup_dict[hero1_in_pair]['heroes_good_with2']:
        good_with_score[i][1] = .5
    if hero2_in_pair in heroes_matchup_dict[hero1_in_pair]['heroes_good_with1']:
        good_with_score[i][1] = 1

In [ ]:
good_with_score1 = good_with_score[:]

for i in range(len(good_with_score1)):

    hero1_in_pair = heroes_matchup_pair[i][0]
    hero2_in_pair = heroes_matchup_pair[i][1]

    similar_pairs = [y for y in good_with_score if hero1_in_pair in y[0] and hero2_in_pair in y[0]]    

    heroes_pair_1 = similar_pairs[0]
    heroes_pair_2 = similar_pairs[1]

    if abs(heroes_pair_1[1]) > abs(heroes_pair_2[1]):
        good_with_score1[good_with_score1.index(heroes_pair_2)][1] = heroes_pair_1[1]
    elif abs(heroes_pair_1[1]) < abs(heroes_pair_2[1]):
        good_with_score1[good_with_score1.index(heroes_pair_1)][1] = heroes_pair_2[1]

## Laning Carry bonus Table

The idea of the table is giving 3-point carry hero a lower growth than 1-point carry at the early stage of the game. The carry growth is exponential and proportional to the natural carry points, meaning a 3-point carry will start slow and eventually outcarry the others

In [ ]:
laning_point_series = list(range(-50, 51, 1))
for i in range(0, 101):
    laning_point_series[i] = round(laning_point_series[i] /10, 1) 

carry1_min_point = 1
carry2_min_point = 1
carry3_min_point = 1
carry_min_point = [carry1_min_point, carry2_min_point, carry3_min_point]

carry1_0_point = 1.5
carry2_0_point = carry1_0_point - 0.2
carry3_0_point = carry2_0_point - 0.2*2
carry_0_point = [carry1_0_point, carry2_0_point, carry3_0_point]

carry1_max_point = 2
carry2_max_point = carry1_max_point - 0.1
carry3_max_point = carry2_max_point - 0.1*2
carry_max_point = [carry1_max_point, carry2_max_point, carry3_max_point]

carry_laning_tbl = list(itertools.product([1,2,3], laning_point_series))

In [ ]:
carry_laning_tbl =  pd.DataFrame(carry_laning_tbl, columns=['natural_carry_point', 'laning_bonus'])
carry_laning_tbl['carry_point_after_laning'] = 0

for i in range(1,4):
    carry_laning_tbl.loc[(carry_laning_tbl['natural_carry_point']==i) & (carry_laning_tbl['laning_bonus']==-5.0), 'carry_point_after_laning'] = carry_min_point[i-1]
for i in range(1,4):
    carry_laning_tbl.loc[(carry_laning_tbl['natural_carry_point']==i) & (carry_laning_tbl['laning_bonus']==0.0), 'carry_point_after_laning'] = carry_0_point[i-1]
for i in range(1,4):
    carry_laning_tbl.loc[(carry_laning_tbl['natural_carry_point']==i) & (carry_laning_tbl['laning_bonus']==5.0), 'carry_point_after_laning'] = carry_max_point[i-1]

carry1_neg5_0 = (carry_laning_tbl.loc[50, 'carry_point_after_laning'] - carry_laning_tbl.loc[0, 'carry_point_after_laning'])/50
for i in range(1, 50):
    carry_laning_tbl.loc[i, 'carry_point_after_laning'] = carry_laning_tbl.loc[i-1, 'carry_point_after_laning'] + carry1_neg5_0

carry1_0_50 = (carry_laning_tbl.loc[100, 'carry_point_after_laning'] - carry_laning_tbl.loc[50, 'carry_point_after_laning'])/50    
for i in range(51, 100):
    carry_laning_tbl.loc[i, 'carry_point_after_laning'] = carry_laning_tbl.loc[i-1, 'carry_point_after_laning'] + carry1_0_50

carry2_neg5_0 = (carry_laning_tbl.loc[151, 'carry_point_after_laning'] - carry_laning_tbl.loc[101, 'carry_point_after_laning'])/50    
for i in range(102, 151):
    carry_laning_tbl.loc[i, 'carry_point_after_laning'] = carry_laning_tbl.loc[i-1, 'carry_point_after_laning'] + carry2_neg5_0

carry2_0_50 = (carry_laning_tbl.loc[201, 'carry_point_after_laning'] - carry_laning_tbl.loc[151, 'carry_point_after_laning'])/50    
for i in range(152, 201):
    carry_laning_tbl.loc[i, 'carry_point_after_laning'] = carry_laning_tbl.loc[i-1, 'carry_point_after_laning'] + carry2_0_50

carry3_neg5_0 = (carry_laning_tbl.loc[252, 'carry_point_after_laning'] - carry_laning_tbl.loc[202, 'carry_point_after_laning'])/50 
for i in range(203, 252):
     carry_laning_tbl.loc[i, 'carry_point_after_laning'] = carry_laning_tbl.loc[i-1, 'carry_point_after_laning'] + carry3_neg5_0
        
carry3_0_50 = (carry_laning_tbl.loc[302, 'carry_point_after_laning'] - carry_laning_tbl.loc[252, 'carry_point_after_laning'])/50 
for i in range(253, 302):
     carry_laning_tbl.loc[i, 'carry_point_after_laning'] = carry_laning_tbl.loc[i-1, 'carry_point_after_laning'] + carry3_0_50

In [ ]:
carry_laning_tbl.to_csv('C:/Users/anhqu/OneDrive/Desktop/QAs/QA github folder/Carries_laning_growth.csv')